<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_10_3_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# T81-558: Applications of Deep Neural Networks
**Module 10: Time Series in Keras**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 10 Material

* Part 10.1: Time Series Data Encoding for Deep Learning [[Video]](https://www.youtube.com/watch?v=dMUmHsktl04&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_1_timeseries.ipynb)
* Part 10.2: Programming LSTM with Keras and TensorFlow [[Video]](https://www.youtube.com/watch?v=wY0dyFgNCgY&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_2_lstm.ipynb)
* **Part 10.3: Text Generation with Keras and TensorFlow** [[Video]](https://www.youtube.com/watch?v=6ORnRAz3gnA&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_3_text_generation.ipynb)
* Part 10.4: Image Captioning with Keras and TensorFlow [[Video]](https://www.youtube.com/watch?v=NmoW_AYWkb4&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_4_captioning.ipynb)
* Part 10.5: Temporal CNN in Keras and TensorFlow [[Video]](https://www.youtube.com/watch?v=i390g8acZwk&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_5_temporal_cnn.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.

In [1]:
try:
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: not using Google CoLab


# Part 10.3: Text Generation with LSTM

Recurrent neural networks are also known for their ability to generate text.  This can allow the output of the neural network to be free-form text.  In this part we will see how an LSTM can be trained on a textual document, such as classic literature, and learn to output new text that appears to be of the same form as the training material.  If you train your LSTM on [Shakespeare](https://en.wikipedia.org/wiki/William_Shakespeare), then it will learn to crank out new prose that is similar to what Shakespeare had written. 

Don't get your hopes up.  Your not going to each your deep neural network to write the next [Pulitzer Prize for Fiction](https://en.wikipedia.org/wiki/Pulitzer_Prize_for_Fiction).  The prose generated by your neural network will be nonsensical.  However, it will usually be nearly grammatically and of a similar style as the source training documents. 

A neural network generating nonsensical text based on literature may not seem terribly useful at first glance.  However, the reason that this technology gets so much interest is that it forms the foundation for many more advanced technologies.  The fact that the LSTM will typically learn human grammar from the source document opens a wide range of possibilities. Similar technology can be used to complete sentences when a user is entering text.  Simply the ability to output free-form text becomes the foundation of many other technologies.  In the next part, we will make use of this technique to create a neural network that can write captions for images to describe what is going on in the image. 

### Additional Information

The following are some of the articles that I found useful putting this section together.

* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* [Keras LSTM Generation Example](https://keras.io/examples/lstm_text_generation/)

### Character-Level Text Generation

There are a number of different approaches to teaching a neural network to output free-form text.  The most basic question is if you wish the neural network to learn at the word or character level.  In many ways, learning at the character level is the more interesting of the two.  The LSTM is learning construct its own words without even being shown what a word is.  We will begin with character-level text generation.  In the next module, we will see how we can use nearly the same technique to operate at the word level.  The automatic captioning that will be implemented in the next module is at the word level.

We begin by importing the needed Python packages and defining the sequence length, named **maxlen**.  Time-series neural networks always accept their input as a fixed length array.  Not all of the sequence might be used, it is common to fill extra elements with zeros.  The text will be divided into sequences of this length and the neural network will be trained to predict what comes after this sequence.

In [2]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re

For this simple example we will train the neural network on the classic children's book [Treasure Island](https://en.wikipedia.org/wiki/Treasure_Island).  We begin by loading this text into a Python string and displaying the first 1,000 characters.

In [3]:
r = requests.get("https://data.heatonresearch.com/data/t81-558/text/"\
                 "treasure_island.txt")
raw_text = r.text
print(raw_text[0:1000])


ï»¿The Project Gutenberg EBook of Treasure Island, by Robert Louis Stevenson

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: Treasure Island

Author: Robert Louis Stevenson

Illustrator: Milo Winter

Release Date: January 12, 2009 [EBook #27780]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK TREASURE ISLAND ***




Produced by Juliet Sutherland, Stephen Blundell and the
Online Distributed Proofreading Team at http://www.pgdp.net









 THE ILLUSTRATED CHILDREN'S LIBRARY


         _Treasure Island_

       Robert Louis Stevenson

          _Illustrated by_
            Milo Winter


           [Illustration]


           GRAMERCY BOOKS
              NEW YORK




 Foreword copyright Â© 1986 by Random House V


We will extract all unique characters from the text and sort them.  This allows us to assign a unique ID to each character.  Because the characters are sorted, these IDs should remain the same.  If new characters were added to the original text, then the IDs would change.  We build up two dictionaries.  The first **char2idx** is used to convert a character into its ID.  The second **idx2char** converts an ID back into its character.

In [4]:
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]',r'', processed_text) 

In [5]:
print('corpus length:', len(processed_text))

chars = sorted(list(set(processed_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 397400
total chars: 60


We are now ready to build the actual sequences.  Just like previous neural networks there will be an $x$ and $y$.  However, for the LSTM, $x$ and $y$ will both be sequences.  The $x$ input will specify the sequences where $y$ are the expected output.  The following code generates all possible sequences.

In [6]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i + maxlen])
    next_chars.append(processed_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 132454


In [7]:
sentences

['the project gutenberg ebook of treasure ',
 ' project gutenberg ebook of treasure isl',
 'oject gutenberg ebook of treasure island',
 'ct gutenberg ebook of treasure island, b',
 'gutenberg ebook of treasure island, by r',
 'enberg ebook of treasure island, by robe',
 'erg ebook of treasure island, by robert ',
 ' ebook of treasure island, by robert lou',
 'ook of treasure island, by robert louis ',
 ' of treasure island, by robert louis ste',
 ' treasure island, by robert louis steven',
 'easure island, by robert louis stevenson',
 'ure island, by robert louis stevenson\r\n\r',
 ' island, by robert louis stevenson\r\n\r\nth',
 'land, by robert louis stevenson\r\n\r\nthis ',
 'd, by robert louis stevenson\r\n\r\nthis ebo',
 'by robert louis stevenson\r\n\r\nthis ebook ',
 'robert louis stevenson\r\n\r\nthis ebook is ',
 'ert louis stevenson\r\n\r\nthis ebook is for',
 ' louis stevenson\r\n\r\nthis ebook is for th',
 'uis stevenson\r\n\r\nthis ebook is for the u',
 ' stevenson\r\n\r\n

In [8]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [9]:
x.shape

(132454, 40, 60)

In [10]:
y.shape

(132454, 60)

The dummy variables for $y$ are shown below.

In [11]:
y[0:10]

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, Fal

Next, we create the neural network.  The primary feature of this neural network is the LSTM layer.  This allows the sequences to be processed.  

In [12]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               96768     
_________________________________________________________________
dense (Dense)                (None, 60)                7740      
Total params: 104,508
Trainable params: 104,508
Non-trainable params: 0
_________________________________________________________________


The LSTM will produce new text character by character.  We will need to sample the correct letter from the LSTM predictions each time.  The **sample** function accepts the following two parameters:

* **preds** - The output neurons.
* **temperature** - 1.0 is the most conservative, 0.0 is the most confident (willing to make spelling and other errors).

The sample function below is essentially performing a [softmax]() on the neural network predictions.  This causes each output neuron to become a probability of its particular letter.  

In [14]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [15]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


We are now ready to train.  It can take up to an hour to train this network, depending on how fast your computer is.  If you have a GPU available, please make sure to use it.

In [ ]:
# Ignore useless W0819 warnings generated by TensorFlow 2.0.  Hopefully can remove this ignore in the future.
# See https://github.com/tensorflow/tensorflow/issues/31308
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Train on 132454 samples
Epoch 1/60
132224/132454 [============================>.] - ETA: 0s - loss: 2.0288****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: " at one side of the
companion; sometime"
 at one side of the
companion; sometime and the shortor of the porting to the postor to the portor the could the porting the bostor his could not the coptain of the shorters in the partion to the porting to the postor to the shorting and the said the doctor and the shorth and the portor to the porters of the shorter and a man to the porting to the said the shorter the shorting the shorting to the doctor to the porting to the prose to t
----- temperature: 0.5
----- Generating with seed: " at one side of the
companion; sometime"
 at one side of the
companion; sometime of the meart. string a sinterone for the shorter as deed to the sand word a phind of the poution a shimbor with th

t only a single shot had been fired. it wasted paised the falt--"i who the he, but it was thet'if thisg that
sill. last hap.

s sitt, liveden, thin. had hit skips."

and thes west
ware, where
when, a cload to the liddnout, i !r iffain a giveser
oard, weth, though
rehowared, she dih mofe, to kee that  currel best
a newfore," cried liked.

"ahal man's loger half."

"gutai'ed, pershing sayoning you, reashifg lyest
thatfoe
b
132454/132454 [==============================] - 118s 889us/sample - loss: 1.3801
Epoch 8/60
131968/132454 [============================>.] - ETA: 0s - loss: 1.3613****************************************************************************
----- Generating text after Epoch: 7
----- temperature: 0.2
----- Generating with seed: " and my heart jumped against my ribs. th"
 and my heart jumped against my ribs. the streather the captain, when i was so long and the stranger and the strong of the stranger and the stranger and the stockade, and the boat and the captain spide o

moonlit avenue to what a safe-buttern
sound, now, "you'll over like a surous," returned the cove, laying, you do see in found them georges was, the evening opy
indooty; there we had clad two to see for hort op? hawkins would good  pake, do to be noo to strim last basth, after as us, and then, when it out in whon a monly, for there is a man's powd out what, i teke our meant upon he paused his letch.

i faits and f
----- temperature: 1.2
----- Generating with seed: "he long, leafless,
moonlit avenue to wh"
he long, leafless,
moonlit avenue to which i lept behint.  sill
sly, you're siud pewaying?" he cried.

ell.s
if you would up
balmorge of anyones," cop an'th; but my who, morgarbsave you?  pown rainin, muaten that so, dies, abother; there is so with caired do-goodistate, if rateed her.
su, we like,ing,
xaing here only he should know , weakersbed, if
do thouch as
by these asho. repent.

terrsclasir state meanting toward rup av
132454/132454 [==============================] - 88s 668us/sa

Epoch 18/60
131968/132454 [============================>.] - ETA: 0s - loss: 1.2672****************************************************************************
----- Generating text after Epoch: 17
----- temperature: 0.2
----- Generating with seed: "aid he thought somebody might read a
pr"
aid he thought somebody might read a
project guther the stockade, and which was a change of the stockade, and a stockade and the sea the stockade and the stockade, and then had a the more and the sea along was the stockade, and a state he was a speak the stockade, and a sunder of the stockade, and a stockade and the stockade, and then he was a good the stockade, and a speak of the sun of the stockade, and a stockade and the stockade, a
----- temperature: 0.5
----- Generating with seed: "aid he thought somebody might read a
pr"
aid he thought somebody might read a
project guther the cat hunter, and a soon all the dead lettle of the project gutenberg sme, and the sea take a boat was a hands--for he was

greature pature,
doy it, and i, paused to obboly all but than i, but nothing my
lait in a sures.

"poeps to 'emle"r
should but it wark.

this was
nithed all."

not his yarsh were in a stoo's bird, 
"held but
motion seige, it -laicits, and is a great bralling, which bore hundredness, and though we were too, about
132454/132454 [==============================] - 118s 893us/sample - loss: 1.2381
Epoch 25/60
131968/132454 [============================>.] - ETA: 0s - loss: 1.2325****************************************************************************
----- Generating text after Epoch: 24
----- temperature: 0.2
----- Generating with seed: " who
had been to field-work on the far "
 who
had been to field-work on the far their had stranged and sailed to the stranger of the captain, which we were stranged of the captain stranged to the captain stands and been the party and the story that was the stranger of the story to the trees, and the strangers and the book of the more and the redruthwas

over complede of an ssull sealts. i have a great useles; the
boats no feet
wors, we must the party, i kild it did to take them.




chapter kie through then all the indericlet like a bible, ah, there grewand to fire publed manders
in over on this
man with his chest, under helfersly,
will obver there, i shugg
mates. if we seemet open like a cheeped at the part very,
he made that
mate thhe
----- temperature: 1.2
----- Generating with seed: "und me were little ripples, combing
ove"
und me were little ripples, combing
overhaul galler in the
dock of shou lute of his yusen direces
dick. you have you. but i permates, john overhee bones, and
were followed, the voice a clabs now run voiter
to, a stood together."

i piece incony
advant plinifelias his ha,
adaltast to my
bloods, when, hurely all sauge teating, i old his on them wold, as
than's shot on its any and tright voice you rise, but
george, stood you ha
132454/132454 [==============================] - 122s 919us/sample - loss: 1.2085
Epoch

hawkins?" he would done the sea, took to mo sailor the side of the stockade, and then i asked the stockade, and a place and strock, he was a south, and the shore of the captain and me. he had taken up the dead broad to the momen sill, and which was the pipe. "he'd the sun he may con't for me, and i thought it with him and he of the inine like should sleep he said, and then he was a barrice, and then i had liked t
----- temperature: 1.0
----- Generating with seed: "were, now, weren't it,
hawkins?" he wou"
were, now, weren't it,
hawkins?" he woulded the squire. i don't comprated well open supers and serjibering for conwass, a app's
nemiston, and a great work all, jact. but 'udder it, and give the swoulst badon so think
nowiry: "plaan doctor befist than
sign i of this three
the last. every gave of this wind of him disegrence would toing
kness before him, a firgs
hup than they could see off. and so he could the
leaming, together, fo
----- temperature: 1.2
----- Generating with seed: "were,

alongside lay one of the gigs adajiney." madessot, turned before it, aerily, tobbess, george reture part ingbogming in the shouldered down was pointiol counse fall intenking voluelles above
ristle my slocked about nem every renth his shore,
and as i redert; blabling varying furthing
the diriccery to little fall
that settary. he rever. we don't between the just meding to passes, more traced helped through the schrieation,
132454/132454 [==============================] - 88s 662us/sample - loss: 1.1803
Epoch 42/60
132352/132454 [============================>.] - ETA: 0s - loss: 1.1753****************************************************************************
----- Generating text after Epoch: 41
----- temperature: 0.2
----- Generating with seed: "as the other.

"the captain's wounded,"
as the other.

"the captain's wounded," said the southouse.

"well, we'll do my hand and the sea-changs to see the corner of the captain and the woods and setted he was stood on the cabin was the shoresti

C:\Users\jheaton\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


over his face, treasure very long from own hands where he thind; 
----- temperature: 1.2
----- Generating with seed: "mart, too," answered silver, shaking
ha"
mart, too," answered silver, shaking
has a way wading ill hear twober swidas. put,
"first."

cever of
to 
 nootine i pitted that we had saw by his conloon and
, this erought act of itsens instoned eyerations wear the
aitvooh risially; an't were of roatide 
they don't more know, black wigh man keep your chim--buly before much."

whether who had been a rud! i
compled.

"well, i're adamt, as i know alo oped a'sing oye--told
132454/132454 [==============================] - 107s 810us/sample - loss: 1.1689
Epoch 46/60
131968/132454 [============================>.] - ETA: 0s - loss: 1.1666****************************************************************************
----- Generating text after Epoch: 45
----- temperature: 0.2
----- Generating with seed: "mpanied their examination, you would hav"
mpanied their examination, you would have 